In [1]:
# Log in
from azureml.core.authentication import InteractiveLoginAuthentication
ia = InteractiveLoginAuthentication(tenant_id='72f988bf-86f1-41af-91ab-2d7cd011db47')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [2]:
from azure.storage.blob import BlobServiceClient
connect_str = 'DefaultEndpointsProtocol=https;AccountName=tinyimagstorage0691ba34b;AccountKey=af3IYOodEqOxfhuZhOj050Skaf16szDbwxzhZa8Uh4T8v0lJSVj8ue6IraKIgBYgkUn8G1zND4DMt5SojBI/sQ==;EndpointSuffix=core.windows.net'
# Create the BlobServiceClient object which will be used to create a container client
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

In [3]:
blob_service_client

In [19]:
from azureml.core import Workspace, Datastore
ws = Workspace.get(name="tiny-image-net",
               subscription_id='92c76a2f-0e1c-4216-b65e-abf7a3f34c1e',
               resource_group='AzureML_UW_ImageClassification')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [20]:
# Get a named datastore from the current workspace
datastore = Datastore.get(ws, datastore_name='tinyimagstorage0691ba34b')

In [21]:
# List all datastores registered in the current workspace
datastores = ws.datastores
for name, datastore in datastores.items():
    print(name, datastore.datastore_type)

tinyimagstorage0691ba34b AzureBlob
myblobdatastore AzureBlob
workspacefilestore AzureFile
workspaceblobstore AzureBlob


In [22]:
datastore = ws.get_default_datastore()

$AZUREML_DATAREFERENCE_tinyimagstorage0691ba34b

In [26]:
import tempfile
mounted_path = tempfile.mkdtemp()

# mount dataset onto the mounted_path of a Linux-based compute
mount_context = datastore.mount(mounted_path)

mount_context.start()

import os
print(os.listdir(mounted_path))
print (mounted_path)

AttributeError: 'AzureBlobDatastore' object has no attribute 'mount'

In [10]:
ds1 = ws.datasets['Tiny ImageNet']

In [11]:
type(ds1)

azureml.data.file_dataset.FileDataset

In [20]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [9]:
#Import all of the needed library
import keras
import numpy as np
from keras import applications
from keras.applications import inception_v3
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import os

#Initiate Google Drive to fetch and save your files and datasets
#drive.mount('/content/gdrive')
#os.chdir('/content/gdrive/My Drive/Colab Notebooks/')

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer

#In the last few years, experts have turned to global average pooling (GAP) layers to minimize overfitting by 
#reducing the total number of parameters in the model. Similar to max pooling layers, GAP layers are used to reduce
#the spatial dimensions of a three-dimensional tensor. However, GAP layers perform a more extreme type of 
#dimensionality reduction, where a tensor with dimensions h×w×d is reduced in size to have dimensions 1×1×d 
#GAP layers reduce each h×w feature map to a single number by simply taking the average of all hw values
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(200, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [10]:
import pandas as pd

target_img_size=(96, 96) # Can change
seed = 0


#1, 2, 3, 4 are temporary column names. These aren't needed and are dropped.
val_data = pd.read_csv("./val/val_annotations.txt", sep="\t",
                  header=None, names=['file', 'class', '1', '2', '3', '4'])
val_data.drop(['1', '2', '3', '4'], axis=1, inplace=True)


#Generate train, validation, and test data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

valid_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

target_img_size=(96, 96)

train_generator = train_datagen.flow_from_directory(
        './train/',
        target_size=target_img_size,
        color_mode='rgb',
        batch_size=64,
        class_mode='categorical')

validation_data_generator = valid_datagen.flow_from_dataframe(val_data,
                                directory="./val/images", 
                                x_col="file", y_col="class",
                                target_size=target_img_size, color_mode="rgb",
                                class_mode="categorical", batch_size=64,
                                shuffle=True, seed=seed)

Found 100000 images belonging to 200 classes.
Found 10000 validated image filenames belonging to 200 classes.


In [11]:
from keras import callbacks
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

earlystop = callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=5,
                                    restore_best_weights=True)
# earlystop = callbacks.ReduceLROnPlateau(mode="min")

model.fit_generator(train_generator, epochs=15, steps_per_epoch=200,
                    validation_steps=200,
                    validation_data=validation_data_generator, verbose=1,
                    callbacks=[earlystop])

Epoch 1/15
200/200 [==============================] - 184s 922ms/step - loss: 4.7024 - val_loss: 3.9693
Epoch 2/15
200/200 [==============================] - 62s 309ms/step - loss: 3.7750
Epoch 3/15
200/200 [==============================] - 66s 328ms/step - loss: 3.5546
Epoch 4/15
200/200 [==============================] - 65s 323ms/step - loss: 3.4540
Epoch 5/15
200/200 [==============================] - 79s 397ms/step - loss: 3.3789
Epoch 6/15
200/200 [==============================] - 71s 357ms/step - loss: 3.3639
Epoch 7/15
200/200 [==============================] - 75s 377ms/step - loss: 3.2681
Epoch 8/15
200/200 [==============================] - 82s 409ms/step - loss: 3.2775
Epoch 9/15
200/200 [==============================] - 110s 548ms/step - loss: 3.2657
Epoch 10/15
200/200 [==============================] - 72s 362ms/step - loss: 3.2764
Epoch 11/15
200/200 [==============================] - 92s 459ms/step - loss: 3.2668
Epoch 12/15
200/200 [==============================] 

OSError: Unable to create file (unable to open file: name = './models/first_model.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 602)

In [13]:
model.save("./inception_model.h5")